In [9]:
import pandas as pd

# Get NASDAQ-100 tickers & sectors from Wikipedia
nasdaq100_url = "https://en.wikipedia.org/wiki/NASDAQ-100"
nasdaq100_table = pd.read_html(nasdaq100_url, header=0)[4]  # Extract table (4th table is NASDAQ-100 list)

# Keep only relevant columns
nasdaq100_data = nasdaq100_table[['Symbol', 'Company', 'GICS Sector']]
nasdaq100_data.columns = ['Ticker', 'Company', 'Sector']

# Display first few rows
print(nasdaq100_data.head())

# Save to CSV (Optional)
nasdaq100_data.to_csv("nasdaq100_tickers.csv", index=False)


  Ticker                  Company                  Sector
0   ADBE               Adobe Inc.  Information Technology
1    AMD   Advanced Micro Devices  Information Technology
2   ABNB                   Airbnb  Consumer Discretionary
3  GOOGL  Alphabet Inc. (Class A)  Communication Services
4   GOOG  Alphabet Inc. (Class C)  Communication Services


In [164]:
import yfinance as yf

# Load NASDAQ-100 tickers
tickers = nasdaq100_data["Ticker"].tolist()

# Define time period
start_date = "2005-01-01"
end_date = "2025-01-01"

# Create dictionary to store stock data
stock_data = pd.DataFrame()
OCHLV = ["Open","Close","High","Low","Volume"]
# Fetch data for each stock
for ticker in tickers:
    try:
        df_ = yf.download(ticker, start=start_date, end=end_date, progress=False)
        temp = df_.reset_index().reset_index()
        stock_df = pd.DataFrame(temp["Date"])
        
        for u in OCHLV:
            t = temp[u][ticker]
            tt = pd.DataFrame({u:t})
            stock_df = pd.concat([stock_df,tt],axis = 1)

        stock_df["Ticker"] = ticker  
        stock_data = pd.concat([stock_data,stock_df])
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

stock_data.to_csv("nasdaq100_stock_prices.csv")


In [167]:
## timestamp 
price_df = pd.read_csv("nasdaq100_stock_prices.csv")
price_df = price_df.merge(nasdaq100_data, on = "Ticker", how = "left")
price_df = price_df.drop(price_df.columns[0], axis=1)
price_df['Date'] = pd.to_datetime(price_df['Date'])
price_df.columns = ["date","open","close","high","low","volume","ticker","company","sector"]
price_df.head()
price_df.to_csv("nasdaq100_stock_prices.csv")

In [147]:
# rolling avg dollar volume 
price_df['dollar_vol'] = price_df[['Close','Volume']].prod(axis = 1)
price_df['dollar_vol_1m'] = (
    price_df.groupby('Ticker')['dollar_vol']
    .rolling(window=21, min_periods=1)  
    .mean()
    .reset_index(level=0, drop=True)  
)
price_df['dollar_vol_rank'] = (price_df.groupby(price_df['Date'])
                             .dollar_vol_1m
                             .rank(ascending=False))

In [156]:
import ta 

price_df['rsi'] = price_df.groupby('Ticker')['Close'].transform(lambda x: ta.momentum.RSIIndicator(close=x).rsi())

# MACD Calculation (Per Ticker)
price_df['macd'] = price_df.groupby('Ticker')['Close'].transform(lambda x: ta.trend.MACD(close=x).macd())

# Bollinger Bands Calculation (Upper, Middle, Lower)
def compute_bbands(x):
    bb = ta.volatility.BollingerBands(close=x)
    return pd.DataFrame({
        'bb_upper': bb.bollinger_hband(),
        'bb_lower': bb.bollinger_lband()
    })

bbands_df = price_df.groupby('Ticker')['Close'].apply(compute_bbands).reset_index(level=0, drop=True)
price_df = pd.concat([price_df, bbands_df], axis=1)  # Merge back

# ATR Calculation (Per Ticker)
price_df['atr'] = (
    price_df.groupby('Ticker')
    .apply(lambda x: ta.volatility.AverageTrueRange(high=x['High'], low=x['Low'], close=x['Close']).average_true_range())
    .reset_index(level=0, drop=True)
)


C:\Users\tjuir\AppData\Local\Temp\ipykernel_26484\523243952.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: ta.volatility.AverageTrueRange(high=x['High'], low=x['Low'], close=x['Close']).average_true_range())


In [155]:
price_df.to_csv("nasdaq100_stock_prices.csv")